In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

In [ ]:
PINECONE_API_KEY = "efaa6023-e740-4024-bce9-6c3ab56b6aeb"
PINECONE_API_ENV = "gcp-starter"

In [3]:
def load_pdf_files(data_directory):
    pdf_loader = DirectoryLoader(
        data_directory,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    docs = pdf_loader.load()

    return docs